##Type-1 : Nominal Encoding 
###### (Number does not have significance)

3types
```
1.One Hot Encoding
```
```
2.One Hot Encoding with Many categories
```
```
3.Mean Encoding
```
```
Weight of Evidense
```

##Type-1 : Ordinal Encoding 
###### (Number do have significance)

2types
```
1.Label Encoding
```
```
2. Target Guided Encoding
```

#### 1. Count or Frequency Encoding (Nominal Data)

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv('mercedes.csv',usecols=['X1','X2'])
df.head()

FileNotFoundError: ignored

In [ ]:
pd.get_dummies(df).shape

(4209, 71)

In [ ]:
df['X1'].nunique()

27

In [ ]:
df_frequency_map=df['X2'].value_counts().to_dict()

In [ ]:
df['X2']=df.X2.map(df_frequency_map)

In [ ]:
df.head()

,X1,X2
0,v,6
1,t,4
2,w,137
3,t,137
4,v,137


### Ordinal Feature Encoding

In [ ]:
import datetime

In [ ]:
df_base=datetime.datetime.today()
df_date_list=[df_base-datetime.timedelta(days=x) 
              for x in range(0,20)]
df=pd.DataFrame(df_date_list)
df.columns=['day']
df

,day
0,2021-03-28 08:43:46.342777
1,2021-03-27 08:43:46.342777
2,2021-03-26 08:43:46.342777
3,2021-03-25 08:43:46.342777
4,2021-03-24 08:43:46.342777
5,2021-03-23 08:43:46.342777
6,2021-03-22 08:43:46.342777
7,2021-03-21 08:43:46.342777
8,2021-03-20 08:43:46.342777
9,2021-03-19 08:43:46.342777


In [ ]:
df['day_of_week']=df['day'].dt.day_name()
df.head()

,day,day_of_week
0,2021-03-28 08:43:46.342777,Sunday
1,2021-03-27 08:43:46.342777,Saturday
2,2021-03-26 08:43:46.342777,Friday
3,2021-03-25 08:43:46.342777,Thursday
4,2021-03-24 08:43:46.342777,Wednesday


In [ ]:
df['day_of_week'].unique()

array(['Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday',
       'Monday'], dtype=object)

In [ ]:
weekday_map={'Sunday':0, 'Saturday':6, 'Friday':5, 'Thursday':4, 'Wednesday':3, 'Tuesday':2,
       'Monday':1}

In [ ]:
df['day_ordinal']=df['day_of_week'].map(weekday_map)

# Target Guided Ordinal Encoding

##### Ordering the labels according to the target

##### Replace the labels by the joint probability of being 1 or 0

In [ ]:
import seaborn as sns
import numpy as np

In [ ]:
df=sns.load_dataset('titanic')
df=df[['survived','deck']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  891 non-null    int64   
 1   deck      203 non-null    category
dtypes: category(1), int64(1)
memory usage: 8.3 KB


In [ ]:
# df['deck']=df['deck'].fillna('Missing')

df['deck'] = df['deck'].cat.add_categories('Missing')
df['deck'].fillna('Missing', inplace =True) 
df['deck'+'_first_letter']=df['deck'].astype(str).str[0]
df.head()
# df['deck'].replace(np.nan,'Un', inplace =True)

,survived,deck,deck_first_letter
0,0,Missing,M
1,1,C,C
2,1,Missing,M
3,1,C,C
4,0,Missing,M


In [ ]:
df.groupby(['deck_first_letter'])['survived'].mean()

deck_first_letter
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
M    0.299419
Name: survived, dtype: float64

In [ ]:
ordinal_labels=df.groupby(['deck_first_letter'])['survived'].mean().sort_values().index
ordinal_labels

Index(['M', 'A', 'G', 'C', 'F', 'B', 'E', 'D'], dtype='object', name='deck_first_letter')

In [ ]:
## 'D' is having the highest mean so it is ranked high
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
ordinal_labels2

{'A': 1, 'B': 5, 'C': 3, 'D': 7, 'E': 6, 'F': 4, 'G': 2, 'M': 0}

# Probability Ratio Encoding

In [ ]:
df=sns.load_dataset('titanic')
df=df[['survived','deck']]

In [ ]:
df['deck'] = df['deck'].cat.add_categories('Missing')
df['deck'].fillna('Missing', inplace =True)
df['deck']=df['deck'].astype(str).str[0] 
df.head()

,survived,deck
0,0,M
1,1,C
2,1,M
3,1,C
4,0,M


In [ ]:
prop_df=df.groupby(['deck'])['survived'].mean()
prop_df=pd.DataFrame(prop_df)
prop_df

,survived
deck,
A,0.466667
B,0.744681
C,0.593220
D,0.757576
E,0.750000
F,0.615385
G,0.500000
M,0.299419


In [ ]:
prop_df['Died']=1-prop_df['survived']
prop_df.head()

,survived,Died
deck,,
A,0.466667,0.533333
B,0.744681,0.255319
C,0.593220,0.406780
D,0.757576,0.242424
E,0.750000,0.250000


In [ ]:
# probability rattio
prop_df['Probability_ratio']=prop_df['survived']/prop_df['Died']
prop_df

,survived,Died,Probability_ratio
deck,,,
A,0.466667,0.533333,0.875000
B,0.744681,0.255319,2.916667
C,0.593220,0.406780,1.458333
D,0.757576,0.242424,3.125000
E,0.750000,0.250000,3.000000
F,0.615385,0.384615,1.600000
G,0.500000,0.500000,1.000000
M,0.299419,0.700581,0.427386


In [ ]:
prob_encoded=prop_df['Probability_ratio'].to_dict()
prob_encoded

{'A': 0.875,
 'B': 2.916666666666666,
 'C': 1.4583333333333333,
 'D': 3.125,
 'E': 3.0,
 'F': 1.6000000000000003,
 'G': 1.0,
 'M': 0.4273858921161825}

In [ ]:
df['Cabin_encoded']=df['deck'].map(prob_encoded)